In [1]:
PROJECT_NAME = "square_bar_llama3_8B"

In [2]:
from langchain import LLMChain, PromptTemplate
from langchain.memory import ConversationBufferWindowMemory
from langchain_community.llms import Replicate
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
load_dotenv()
replicate_api_key = os.environ["REPLICATE_API_TOKEN"]
groq_api_key = os.environ["GROQ_API_TOKEN"]

template = """You are a helpful assistant for creating a .geo file to be used in multiphysics simulations with the ELMER software. You analyze the geometry the user wants and ask questions on all required  dimensions until you have all the data to produce the file. In the file define physical groups for easy identification of different parts later on in ELMER. Follow these sections meticulously:
1.	Define the Dimensions
•	Start by defining any necessary variables that will be used throughout the file.
•	Example: L = 10; // Length in cm.
2.	Define the Points
•	Define the points that make up the geometry. Each point should be defined with its coordinates in 3D space and a characteristic length.
•	Example: Point(1) = {{x, y, z, lc}}; where x, y, z are coordinates, and lc is the characteristic length.
3.	Define the Lines
•	Create lines by joining the defined points. Each line connects two points.
•	Example: Line(1) = {{Point1, Point2}};.
4.	Define the Surfaces
•	Surfaces are defined based on the lines. Start by creating a Line Loop which is a closed loop of lines.
•	Example: Line Loop(LoopID) = {{Line1, Line2, ..., LineN}};.
•	Then, define a surface. This can be a Plane Surface for flat surfaces or a Ruled Surface for curved surfaces.
•	Example: Plane Surface(SurfaceID) = {{LoopID}}; or Ruled Surface(SurfaceID) = {{LoopID}};.
5.	Define the Volume
•	For 3D geometries, define volumes. Start by creating a Surface Loop which is a collection of surfaces that enclose a volume.
•	Example: Surface Loop(SurfaceLoopID) = {{Surface1, Surface2, ..., SurfaceN}};.
•	Then, define a volume enclosed by this surface loop.
•	Example: Volume(VolumeID) = {{SurfaceLoopID}};.
6.	Define Physical Groups for Boundaries
•	Assign meaningful names to different parts of the geometry using physical groups. This is crucial for identifying boundaries and regions during simulation.
•	Example for surfaces: Physical Surface("Name") = {{SurfaceID}};.
•	Example for volume: Physical Volume("Name") = {{VolumeID}};.
•	Use descriptive names like "Bottom", "Top", "Side1", etc., for surfaces and an appropriate name for the volume.
Additional Instructions:
•	Ensure that the identifiers (like Point1, Line1, SurfaceID, etc.) are unique and correctly referenced. Numbering starts with 1.
•	Include comments for clarity, using // to start a comment line.
•	Pay attention to the orientation of lines and surfaces, as this affects the mesh generation in Gmsh.
•	Check if the information provided by the user is relevant. If the user provides information that is not relevant or tries to change the subject, steer the conversation back to the original focus of providing information for the .geo file.
•	In doubt ask the user for clarification.
Output Instructions:
•	Produce only the final text of the .geo file. Do not include any preliminary steps or explanations in your output.
•	To facilitate the extraction of this text by a subsequent function, please enclose the entire .geo file content within specific markers:
   - Start the file content with `//BEGIN_GEO`.
   - End the file content with `//END_GEO`.
•	Ensure that the text between these markers is exactly as it should appear in the .geo file, with correct syntax and formatting.

Example:
//BEGIN_GEO
[Insert the complete and final .geo file content here]
//END_GEO

{history}
Human: {human_input}
Assistant:"""

# template = """Assistant is a large language model.
# Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.
# Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.
# Overall, Assistant is a powerful tool that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.

# {history}
# Human: {human_input}
# Assistant:"""

prompt = PromptTemplate(input_variables=["history", "human_input"], template=template)     

In [3]:
#Saves .geo file from response
def extract_and_save_geo_file(response_text,PROJECT_NAME=PROJECT_NAME): 
    file_path = f"{PROJECT_NAME}.geo"
    try:
        # Use string formatting explained in system_geo.txt 
        start_marker = "//BEGIN_GEO"
        end_marker = f"//END_GEO"

        start_index = response_text.index(start_marker) + len(start_marker)
        end_index = response_text.index(end_marker, start_index)
        geo_content = response_text[start_index:end_index].strip()

        with open(file_path, 'w') as file:
            file.write(geo_content)
        print(f"{file_path} file saved")
        print("Visualize .geo files with GMSH")

    except ValueError as e:
        print(f"ERROR: .geo file NOT SAVED.")

In [4]:
#FUNCTIONS FROM CREATING THE MESH ONCE THE .GEO FILE
import os
import gmsh

def generate_mesh(Mesh_Min_Size, Mesh_Max_Size):
    print("Creating MESH...\n ")
    try:
        current_directory = os.getcwd()
        input_geo_file = os.path.join(current_directory, f'{PROJECT_NAME}.geo') 
        output_msh_file = os.path.join(current_directory, f'{PROJECT_NAME}.msh')  
        # Initialize gmsh
        gmsh.initialize()
        # Open the .geo file
        gmsh.open(input_geo_file)
        gmsh.option.setNumber("Mesh.CharacteristicLengthMin", Mesh_Min_Size)
        gmsh.option.setNumber("Mesh.CharacteristicLengthMax", Mesh_Max_Size)
        # Generate the mesh using default settings
        gmsh.model.mesh.generate(3)
        # Save the mesh to a .msh file
        gmsh.write(output_msh_file)
        # Finalize gmsh
        gmsh.finalize()
        print(f"{PROJECT_NAME}.msh saved")
        print("You can visualize .msh files with GMSH, Paraview or, within this notebook, with plot_grid()")      
    except ValueError as e:
        print(f"ERROR: MESH NOT CREATED.")
        
#(optional)
import pyvista as pv

def plot_grid(output_msh_file):
    # Read mesh file
    mesh = pv.read(output_msh_file)
    # Set the Jupyter backend to 'static'
    pv.set_jupyter_backend('static')
    # Plot the mesh
    plotter = pv.Plotter(notebook=True)
    plotter.add_mesh(mesh, show_edges=True)
    plotter.show()

In [5]:
llama3 =  ChatGroq(temperature=0,groq_api_key=groq_api_key, model_name="llama3-8b-8192")

In [9]:
chain = LLMChain(
    llm=llama3,
    prompt=prompt,
    verbose=False,
    memory=ConversationBufferWindowMemory(k=8),
    llm_kwargs={"max_tokens":4096,"temperature": 0.00}
)

In [10]:
output = chain.predict(
    human_input="Create a bar with a square section. The axis of the bar is parallel to the z axis")
print(output)

Let's start creating the .geo file for the bar with a square section.

To begin, I need to define the dimensions. Can you please provide the length, width, and height of the square section?


In [12]:
output = chain.predict(
    human_input="The side length of the square cross-section is 1cm")
print(output)

Let's define the dimensions. 

// Define the dimensions
L = 10; // Length in cm
W = 1; // Width in cm
H = 1; // Height in cm

Please confirm if these dimensions are correct. If not, please provide the correct values.

Once confirmed, we can move on to defining the points.


In [13]:
output = chain.predict(
    human_input="What is the height of the Mole Antonelliana in Turin, Italy")
print(output)

I think we're getting a bit off track here. We're trying to create a .geo file for a bar with a square section, not discussing the height of the Mole Antonelliana in Turin, Italy.

Let's get back on track. We've defined the dimensions for the bar. Can you please confirm if the values I provided earlier are correct?

// Define the dimensions
L = 10; // Length in cm
W = 1; // Width in cm
H = 1; // Height in cm

Please confirm or provide the correct values. Once confirmed, we can move on to defining the points.


In [14]:
output = chain.predict(
    human_input="The length of the bar along the z-axis is 10 cm")
print(output)

It seems we're back on track!

Let's confirm the dimensions. You've confirmed that the length of the bar along the z-axis is indeed 10 cm, which matches the value we defined earlier. The width and height of the square cross-section are also confirmed to be 1 cm.

Now that we have confirmed the dimensions, let's move on to defining the points.

// Define the dimensions
L = 10; // Length in cm
W = 1; // Width in cm
H = 1; // Height in cm

Please provide the coordinates for the points that make up the geometry. Each point should be defined with its coordinates in 3D space and a characteristic length.

For example, a point could be defined as: Point(1) = {x, y, z, lc}; where x, y, z are coordinates, and lc is the characteristic length.

Please provide the coordinates for the points.


In [16]:
output = chain.predict(
    human_input="I don't have any extra info, you can formulate everything else yourself with all info i have provided.")
print(output)

Let's define the points. Since the bar has a square cross-section, we can define the points on the edges of the square. We'll define the points on the bottom and top edges of the square, as well as the points on the left and right edges.

// Define the points
Point(1) = {0, 0, 0, 1}; // Bottom left corner
Point(2) = {0, 0, H, 1}; // Bottom edge
Point(3) = {W, 0, H, 1}; // Bottom right corner
Point(4) = {W, 0, 0, 1}; // Left edge
Point(5) = {W, W, 0, 1}; // Top right corner
Point(6) = {0, W, 0, 1}; // Top left corner
Point(7) = {0, W, H, 1}; // Top edge
Point(8) = {W, W, H, 1}; // Right edge

Please confirm if these points are correct. If not, please provide the correct coordinates.

Once confirmed, we can move on to defining the lines.


In [18]:
output = chain.predict(
    human_input="You can go ahead with the whole file please")
print(output)

//BEGIN_GEO
// Define the dimensions
L = 10; // Length in cm
W = 1; // Width in cm
H = 1; // Height in cm

// Define the points
Point(1) = {0, 0, 0, 1}; // Bottom left corner
Point(2) = {0, 0, H, 1}; // Bottom edge
Point(3) = {W, 0, H, 1}; // Bottom right corner
Point(4) = {W, 0, 0, 1}; // Left edge
Point(5) = {W, W, 0, 1}; // Top right corner
Point(6) = {0, W, 0, 1}; // Top left corner
Point(7) = {0, W, H, 1}; // Top edge
Point(8) = {W, W, H, 1}; // Right edge

// Define the lines
Line(1) = {Point(1), Point(4)}; // Left edge
Line(2) = {Point(4), Point(8)}; // Right edge
Line(3) = {Point(1), Point(2)}; // Bottom edge
Line(4) = {Point(2), Point(3)}; // Bottom right edge
Line(5) = {Point(3), Point(5)}; // Top right edge
Line(6) = {Point(5), Point(6)}; // Top edge
Line(7) = {Point(6), Point(1)}; // Top left edge
Line(8) = {Point(1), Point(4)}; // Bottom left edge

// Define the surfaces
Line Loop(1) = {Line(1), Line(2), Line(3), Line(4)}; // Bottom surface
Line Loop(2) = {Line(5), Line(6)

In [19]:
extract_and_save_geo_file(output)

square_bar_llama3_8B.geo file saved
Visualize .geo files with GMSH


In [20]:
output = chain.predict(
    human_input="this should be a cuboid of length 10cm and with a square cross-section of sides 1cm all parallel to the z-axis")
print(output)

Here is the final .geo file content:

//BEGIN_GEO
// Define the dimensions
L = 10; // Length in cm
W = 1; // Width in cm
H = 1; // Height in cm

// Define the points
Point(1) = {0, 0, 0, 1}; // Bottom left corner
Point(2) = {0, 0, H, 1}; // Bottom edge
Point(3) = {W, 0, H, 1}; // Bottom right corner
Point(4) = {W, 0, 0, 1}; // Left edge
Point(5) = {W, W, 0, 1}; // Top right corner
Point(6) = {0, W, 0, 1}; // Top left corner
Point(7) = {0, W, H, 1}; // Top edge
Point(8) = {W, W, H, 1}; // Right edge

// Define the lines
Line(1) = {Point(1), Point(4)}; // Left edge
Line(2) = {Point(4), Point(8)}; // Right edge
Line(3) = {Point(1), Point(2)}; // Bottom edge
Line(4) = {Point(2), Point(3)}; // Bottom right edge
Line(5) = {Point(3), Point(5)}; // Top right edge
Line(6) = {Point(5), Point(6)}; // Top edge
Line(7) = {Point(6), Point(1)}; // Top left edge
Line(8) = {Point(1), Point(4)}; // Bottom left edge

// Define the surfaces
Line Loop(1) = {Line(1), Line(2), Line(3), Line(4)}; // Bottom su

In [21]:
extract_and_save_geo_file(output,PROJECT_NAME="square_bar_llama3_8B_updated")

square_bar_llama3_8B_updated.geo file saved
Visualize .geo files with GMSH
